# Optimizing Strategy Parameters

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [11]:
import pandas as pd

df = pd.read_excel('data/microsoft-linkedin-processed.xlsx', index_col=0)
df.index = pd.to_datetime(df.index)
df

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction
Price,,,,,,,
2016-12-01,59.200001,60.150002,58.939999,60.110001,34542100,0.084387,UP
2016-12-02,59.250000,59.470001,58.799999,59.080002,25515700,1.610763,UP
2016-12-05,60.220001,60.590000,59.560001,59.700001,23552700,-0.450376,DOWN
2016-12-06,59.950001,60.459999,59.799999,60.430000,19907000,2.313831,UP
2016-12-07,61.369999,61.380001,59.799999,60.009998,30809000,-0.590068,DOWN
...,...,...,...,...,...,...,...
2024-11-04,408.459991,410.420013,405.570007,409.799988,19672300,0.729111,UP
2024-11-05,411.459991,414.899994,408.079987,408.369995,17626000,2.075301,UP
2024-11-06,420.179993,420.450012,410.519989,412.420013,26681800,1.234046,UP


# Simple Investment Strategy

### Create Strategy class

In [12]:
from backtesting import Strategy, Backtest

In [13]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [14]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [15]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [16]:
model_dt.predict(df_explanatory)

array([ 0.2068337 ,  0.08195771,  0.08195771, ..., -1.82704124,
        0.2047183 ,  0.2047183 ])

In [17]:
results = bt.run()

### Interpret backtesting results

In [18]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-01 00:00:00
End,2024-11-08 00:00:00
Duration,2899 days 00:00:00
Exposure Time [%],93.393393
Equity Final [$],60206.022395
Equity Peak [$],66161.861018
Return [%],502.060224


## Parametrize the Investment Strategy

### Create Strategy class

In [19]:
class Regression(Strategy):
    
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [20]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [21]:
list_limits_buy = list(range(0, 11, 1))

In [22]:
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [23]:
list_limits_sell = list(range(0, -11, -1))

In [24]:
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [25]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

CPU times: user 5.78 s, sys: 60.7 ms, total: 5.84 s
Wall time: 1min 13s


### [ ] Interpret backtesting results

In [26]:
results_heatmap = results[1]

In [27]:
df_results_heatmap = results_heatmap.reset_index()

In [28]:
dff = df_results_heatmap.pivot(
    index='limit_buy', columns='limit_sell', values='Return [%]')

In [29]:
dff

limit_sell,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
limit_buy,,,,,,,,,,,
0,534.404942,534.404942,534.404942,513.009177,513.009177,621.189527,637.195861,1346.451636,1548.807680,1811.257745,1275.384090
1,440.522851,440.522851,440.522851,420.332485,420.332485,502.060224,343.915345,815.548862,929.835572,347.709801,186.480930
2,245.598865,245.598865,245.598865,233.107528,233.107528,181.618683,84.401537,207.044661,189.185029,-64.588169,-64.445998
3,101.275701,101.275701,101.275701,93.185690,93.185690,106.150537,45.198261,23.560920,17.697565,-77.953769,-74.361076
4,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
5,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
6,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
7,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
8,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000


In [30]:
dff.sort_index(axis=1, ascending=False)

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,1275.384090,1811.257745,1548.807680,1346.451636,637.195861,621.189527,513.009177,513.009177,534.404942,534.404942,534.404942
1,186.480930,347.709801,929.835572,815.548862,343.915345,502.060224,420.332485,420.332485,440.522851,440.522851,440.522851
2,-64.445998,-64.588169,189.185029,207.044661,84.401537,181.618683,233.107528,233.107528,245.598865,245.598865,245.598865
3,-74.361076,-77.953769,17.697565,23.560920,45.198261,106.150537,93.185690,93.185690,101.275701,101.275701,101.275701
4,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
5,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
6,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
7,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
8,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000


## DataFrame heatmaps for better reporting

In [31]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient()

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,1275,1811,1549,1346,637,621,513,513,534,534,534
1,186,348,930,816,344,502,420,420,441,441,441
2,-64,-65,189,207,84,182,233,233,246,246,246
3,-74,-78,18,24,45,106,93,93,101,101,101
4,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
5,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
6,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
7,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [32]:
import numpy as np

In [33]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,1275,1811,1549,1346,637,621,513,513,534,534,534
1,186,348,930,816,344,502,420,420,441,441,441
2,-64,-65,189,207,84,182,233,233,246,246,246
3,-74,-78,18,24,45,106,93,93,101,101,101
4,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
5,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
6,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
7,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100
